<h1 style="text-align:center;"><strong>ASSIGNMENT 3</strong></h1>


<h4 style="text-align:center;"><strong>MOD300: Mandatory project 3</strong></h4>

<h4 style="text-align:center;"><strong>Group Members: Emad Omar Mohamed, Hassan Nehad Adnan, Sture Odin Domingos Troli</strong></h4>

<h5 style="text-align:center;">Nov 9, 2025</h5>

**Statement from the Group Members:**
We used limited AI assistance in this project, mostly to debug code (for the functions, classes, and specially plotting because it was crashing alot or even would run for very long) and to format mathematical formulas so it would appear when the Markdown would run.

More specifically our implementation is in Python using NumPy for random sampling and array operations, and Matplotlib for plotting. We used free versions of Copilot and Gemini only to help debug minor issues (e.g., vectorization, loops) and to fix small grammar mistakes in this report.

We took some inspiration from Enrico’s lecture codes and posted materials. We used Gemini to typeset formulas and Copilot to improve grammar and clarity, as English is our second language. We also aimed to keep the notebook as short as possible, following the project description: “The shorter the Jupyter notebook in terms of number of lines, the more bonus points you will get (up to 5 points). Comments and assert statements do not count in the line numbers.”

We structured the report with an abstract, introduction, brief explanations of each task and topic, a conclusion, and a summary of each member’s contributions—consistent with the previous two projects—while keeping the length relatively short witout losing overall quality.

<h4 style="text-align:center;"><strong>Abstract</strong></h4>
In this project, we used Monte Carlo simulations to find the volume of a DNA molecule. We first used the method on simple spheres, then added steps so it could also work for more complex molecules. For a sphere, we put random points in a 3D box and check if each point is inside or outside the sphere. The share of points inside tells us the sphere’s volume. We checked that the method works by working out π (pi) and comparing it to the known value.

After that, we used the same method on DNA. We treated each atom as a sphere with a known position and a van der Waals radius from the periodic table. Our results show that Monte Carlo simulations can find volumes well when we use enough random points. We also used a random-walk method to measure the open space around the DNA. These results show that Monte Carlo methods are useful for tough shape problems in molecular biology. We also talk about how close our results are to the real answer, where errors might come from, and how we could make the method better in the future.

#### **TOPIC 1: Calculate DNA volume via Monte Carlo simulation.**



#### **TOPIC 2: Random walk for accessible volume calculation**

### Task 1: Make walkers that move randomly in 3D from different starting points
Random walks explore space differently from Monte Carlo. Monte Carlo samples evenly; random walks mimic particles moving around [4,5].

**What is a random walk?**
A 3D random walk moves one step at a time in a random direction. Each step chooses one of 6 directions: (±x, ±y, ±z) all equally likely [4,5]. This is like how molecules spread in a liquid [8].

**Math idea:**
For random walks, the average squared distance grows linearly with time (steps) [3,8]:

$$\langle r^2(t) \rangle = 6Dt \quad \text{(3D)}$$

where **D** is the diffusion constant and **t** is the number of steps.

**Why use random walks here?**
They can map the open space around DNA by:
- Starting at random points,
- Moving step by step while avoiding DNA atoms,
- Marking which areas are reachable,
- Helping us figure out the open (not occupied) volume [5,8].


## Conclusion and Discussion

### What did we learn?

This project demonstrated how Monte Carlo methods calculate volumes of complex molecular structures. Starting with simple spheres and progressing to real DNA data, we learned that random sampling can solve problems difficult or impossible to address with analytical equations.

**Key insights**

The hit-and-miss Monte Carlo method proved effective for complex geometry: generate random points, check if inside shapes, and calculate volume from the fraction inside. Error decreases as 1/√N, meaning quadrupling the sample size halves the error. For DNA with 259 atoms, we achieved accurate results (~3,266 Å³) with sufficient sampling.

Vectorization provided dramatic performance improvements—from ~100 steps/second to ~1,800,000 steps/second, a 10,000× speedup. This demonstrated that algorithm design is crucial for scientific computing beyond mere correctness.

Van der Waals radii proved essential for molecular volume calculations. Each atom type (H, C, N, O, P) has a specific radius, and DNA atoms overlap significantly (~11% in our calculation), which correctly reflects covalent bonding. The accessible volume (~31,950 Å³) was approximately 10× larger than DNA volume, showing DNA occupies only ~9% of the simulation box—revealing ample space for water, ions, and molecular interactions.

**Real-world relevance**

With real DNA coordinates, we transformed molecular data into useful measurements. These volumes inform understanding of molecular packing, drug binding, protein interactions, and molecular recognition. The method generalizes to any molecule—proteins, RNA, or drug compounds.

### Limitations and improvements

**Model assumptions:** We assumed hard spheres with van der Waals radii, ignoring electron cloud distributions, solvent effects, electrostatics, and molecular flexibility. Real DNA bends and moves in solution.

**Parameter sensitivity:** Different radius tables give varying values, and a 10% change in radii can alter volume by ~30% (V ∝ r³). With 259 atoms and 200,000 sample points, we performed ~52 million distance checks—spatial partitioning algorithms could improve efficiency.

**Static structure:** We used a single DNA configuration, missing conformational flexibility that affects both DNA and accessible volumes. We verified results using volume conservation and convergence but lacked experimental validation from X-ray or NMR data.

**Possible improvements:**
* Multiple DNA configurations from molecular dynamics for average volumes
* Spatial indexing (octrees, k-d trees) for faster point-in-sphere checks
* Explicit or implicit solvent models for solvation effects
* Testing with multiple radius sets (Bondi, Alvarez, Cambridge)
* Uncertainty quantification through bootstrap or Bayesian methods
* 3D interactive visualization of DNA structure and accessible regions

### Broader perspective

Simple models are powerful yet limited. Monte Carlo can calculate volumes for any molecule, but real molecular interactions involve quantum mechanics, electrostatics, entropy, and dynamics. As with all models: some are useful, though none are perfect.

This project combined general mathematical tools with molecule-specific details, requiring both computational skills and chemical knowledge. Real drug discovery combines many methods—geometric calculations, molecular dynamics, quantum chemistry, and experiments—not Monte Carlo alone.

### Final reflections

The progression from simple spheres to DNA made Monte Carlo intuitive while reminding us that behind each coordinate lies a complex molecule essential for life. We witnessed computational progress through vectorization but recognized challenges remain: large proteins have thousands of atoms requiring better force fields and sampling methods.

Skills practiced here—implementing algorithms, optimizing code, validating results, interpreting molecular data, and acknowledging limitations—apply broadly to materials science, drug design, protein folding, and future computational challenges.

**In summary:** We calculated DNA volume (~3,266 Å³) and accessible volume (~31,950 Å³) using Monte Carlo simulation. The main lesson is that Monte Carlo provides powerful tools for molecular calculations, but effective use requires combining computational skills with chemistry, biology, and respect for molecular complexity.

### Group Member Contributions

**Hassan Nehad Adnan:** [text here]

**Emad Omar Mohamed:** [text here]

**Sture Odin Domingos Troli:** Led all Topic 2 tasks, developing both basic and vectorized random walk implementations, designing the accessible volume strategy, and integrating verification tests.

All members collaborated on class and function design, debugging, testing, and writing the abstract, introduction, and conclusion. We worked as a team, helping each other when tasks became challenging, ensuring a cohesive and well-integrated final project.


## Individual Reflections

### Emad Omar Mohamed
[text here]


### Hassan Nehad Adnan
[text here]


### Sture Odin Domingos Troli

Understanding the math—why Monte Carlo error scales like 1/√N and how the Central Limit Theorem fits in—was hard at first, but empowering once it clicked.

Deriving V = (fraction_inside) × V_box was straightforward. Seeing it work on real DNA data was especially satisfying.

Clean plots with good labels made convergence clear. As N grew, the volume estimates settled down, which confirmed the method.

Debugging was tricky. The code sometimes ran without crashing but gave wrong volumes (like negative accessible volume or DNA bigger than the box). Adding assertions and sanity checks for intermediate steps—not just final results—was essential.

The biology piece was tough with our limited chemistry background. Deciding if numbers were reasonabl (is 3,266 Å³ reasonable? what does 11% overlap mean?) needed references and group discussion. It showed why mixing disciplines matters.

We handled Topic 1 well, but Topic 2 was rushed. Next time, we should start the random walk and accessible volume parts earlier.

Suggestions: The math level and pacing were good. The DNA example was engaging. Adding checkpoints or expected value ranges would help groups see if they’re on track sooner. Overall, this felt like real computational chemistry work.

### Collective Reflection

As a group, we're proud of our accomplishments: building and testing Monte Carlo methods, verifying them against theory, applying them to real DNA data, and calculating both occupied and accessible volumes. We gained practical skills in Python for scientific computing, code optimization through vectorization, professional plotting, and collaboration.

Good teamwork requires clear communication, flexible roles, and patience when helping each other. This project needs better time planning—we can improve next time.

This project connected abstract mathematics to real molecular science. The same Monte Carlo methods working for both simple spheres and DNA calculations demonstrated the power and generality of these tools, while reminding us that real molecules are more complex than simple sphere models.

**If we repeated this project, we would:**
- Start earlier, especially on Topic 2
- Write modular, reusable code from the beginning
- Add systematic tests and validation checks
- Plan time better across tasks, reserving adequate verification time
- Try extensions like spatial indexing or different radius sets
- Test with other molecules (proteins, RNA) to verify code generalizability

**Acknowledgments:** We thank the instructors for an engaging project. The mix of clear mathematical goals, real molecular data, logical task progression, and appropriate scope created an excellent learning experience. The balance between simple tasks and complex applications was well-designed, making the project accessible for students from different backgrounds.

This project better prepared us for computational chemistry, molecular modeling, and scientific programming. Understanding Monte Carlo methods, vectorization, and molecular representations through hands-on experience made concepts much clearer than reading alone would have achieved.

#### **References**
